In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [2]:
def gen_traindata():
    data = np.load("./Lorenz.npz")
    return data["t"], data["y"]

In [3]:
N_inside = 500
N_bound = 10
inside_t = lhs(1,N_inside)*(3-0)
bound_t = np.zeros((N_bound,1))
real_bound = np.concatenate((-8*np.ones((N_bound,1)),7*np.ones((N_bound,1)),27*np.ones((N_bound,1))),axis=1)

inside_t = torch.from_numpy(inside_t).float()
bound_t = torch.from_numpy(bound_t).float()
real_bound = torch.from_numpy(real_bound).float()

inside_t.requires_grad_()

###########GPU###########
inside_t = inside_t.cuda()
bound_t = bound_t.cuda()
real_bound = real_bound.cuda()
###########GPU###########

In [4]:
train_t,train_data = gen_traindata()

train_t = torch.from_numpy(train_t).float()
train_data = torch.from_numpy(train_data).float()

n=12
np.random.seed(5678)
index_a = np.random.choice(np.arange(25,dtype=int),n,replace=False)
train_t = train_t[index_a]
train_data = train_data[index_a]

train_t.requires_grad_()

###########GPU###########
train_t = train_t.cuda()
train_data = train_data.cuda()
###########GPU###########

In [5]:
train_t.shape

torch.Size([12, 1])

In [6]:
C1 = torch.tensor(1.0, requires_grad=True)
C2 = torch.tensor(1.0, requires_grad=True)
C3 = torch.tensor(1.0, requires_grad=True)

## $\text{PINN}^{\S}$

In [7]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)


PINNs1 = NN_H2(1, 40, 2, 3)
PINNs1.cuda()

# PINNs1.apply(weights_init)

import torch.nn.init as init
for name, param in PINNs1.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)


optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

optimizer1.add_param_group({'params': [C1,C2,C3], 'lr': 0.001})

nIter1 = 10000

loss_all_1 = []
test_loss_1 = []
C1_a_list = []
C2_a_list = []
C3_a_list = []

loss1_value = 1
it = 0

while  it<20000:
    ##### loss_Bi  ######
    E_bound = PINNs1(bound_t)
    loss_bound = torch.mean(torch.square(E_bound[:,0:1]-real_bound[:,0:1]))+\
                 torch.mean(torch.square(E_bound[:,1:2]-real_bound[:,1:2]))+\
                 torch.mean(torch.square(E_bound[:,2:3]-real_bound[:,2:3]))
    
    ##### loss f  ######    
    E_inside = PINNs1(inside_t)
    E_inside_x = E_inside[:,0:1]
    E_inside_y = E_inside[:,1:2]   
    E_inside_z = E_inside[:,2:3]   
    
    d_x_t = autograd.grad(outputs=E_inside_x, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    
    d_y_t = autograd.grad(outputs=E_inside_y, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    
    d_z_t = autograd.grad(outputs=E_inside_z, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_z),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]      

    loss_f = torch.mean(torch.square(d_x_t-(C1*(E_inside_y-E_inside_x))))+\
             torch.mean(torch.square(d_y_t-(E_inside_x*(C2-E_inside_z)-E_inside_y)))+\
             torch.mean(torch.square(d_z_t-(E_inside_x*E_inside_y-C3*E_inside_z)))
    
    ##### loss observation  ######        
    E_observation = PINNs1(train_t)
    
    loss_observation = torch.mean(torch.square(E_observation[:,0:1]-train_data[:,0:1]))+\
                       torch.mean(torch.square(E_observation[:,1:2]-train_data[:,1:2]))+\
                       torch.mean(torch.square(E_observation[:,2:3]-train_data[:,2:3]))

    loss = loss_bound+loss_f+2*loss_observation
    
    loss_all_1.append(loss.item())
    loss1_value = loss.item()
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    
    if (it+1) % 1000 == 0 or it==0:
        print('It:', it, 'Loss:', loss.item())

        N_inside = 500
        N_bound = 10
        inside_t = lhs(1,N_inside)*(3-0)
        bound_t = np.zeros((N_bound,1))
        real_bound = np.concatenate((-8*np.ones((N_bound,1)),7*np.ones((N_bound,1)),27*np.ones((N_bound,1))),axis=1)

        inside_t = torch.from_numpy(inside_t).float()
        bound_t = torch.from_numpy(bound_t).float()
        real_bound = torch.from_numpy(real_bound).float()

        inside_t.requires_grad_()

        ###########GPU###########
        inside_t = inside_t.cuda()
        bound_t = bound_t.cuda()
        real_bound = real_bound.cuda()
        ###########GPU###########
        
        C1_a_list.append(C1.detach().item())
        C2_a_list.append(C2.detach().item())
        C3_a_list.append(C3.detach().item())
        print('C1:',C1,'C2:',C2,'C3:',C3)
        
    it = it + 1        
loss1_value    

It: 0 Loss: 1519.5653076171875
C1: tensor(0.9990, requires_grad=True) C2: tensor(0.9990, requires_grad=True) C3: tensor(0.9990, requires_grad=True)
It: 999 Loss: 149.2056427001953
C1: tensor(0.2343, requires_grad=True) C2: tensor(1.6374, requires_grad=True) C3: tensor(0.2917, requires_grad=True)
It: 1999 Loss: 120.7764663696289
C1: tensor(0.4264, requires_grad=True) C2: tensor(2.8392, requires_grad=True) C3: tensor(0.1284, requires_grad=True)
It: 2999 Loss: 110.8579330444336
C1: tensor(0.5491, requires_grad=True) C2: tensor(4.0375, requires_grad=True) C3: tensor(0.0733, requires_grad=True)
It: 3999 Loss: 105.59813690185547
C1: tensor(0.6631, requires_grad=True) C2: tensor(5.1905, requires_grad=True) C3: tensor(0.0614, requires_grad=True)
It: 4999 Loss: 100.8341293334961
C1: tensor(0.7422, requires_grad=True) C2: tensor(6.3110, requires_grad=True) C3: tensor(0.0555, requires_grad=True)
It: 5999 Loss: 96.95332336425781
C1: tensor(0.8161, requires_grad=True) C2: tensor(7.4144, requires_gr

0.02172194980084896

## $\text{GA-PINN}^{\S}$

In [9]:
N_inside = 500
N_bound = 10
inside_t = lhs(1,N_inside)*(3-0)
bound_t = np.zeros((N_bound,1))
real_bound = np.concatenate((-8*np.ones((N_bound,1)),7*np.ones((N_bound,1)),27*np.ones((N_bound,1))),axis=1)

inside_t = torch.from_numpy(inside_t).float()
bound_t = torch.from_numpy(bound_t).float()
real_bound = torch.from_numpy(real_bound).float()

inside_t.requires_grad_()

###########GPU###########
inside_t = inside_t.cuda()
bound_t = bound_t.cuda()
real_bound = real_bound.cuda()
###########GPU###########

In [10]:
C1_b = torch.tensor(1.0, requires_grad=True)
C2_b = torch.tensor(1.0, requires_grad=True)
C3_b = torch.tensor(1.0, requires_grad=True)

In [11]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs2 = NN_H2(1, 40, 2, 3)
PINNs2.cuda()

# PINNs1.apply(weights_init)

import torch.nn.init as init
for name, param in PINNs2.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)


optimizer1 = optim.Adam(PINNs2.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

optimizer1.add_param_group({'params': [C1_b,C2_b,C3_b], 'lr': 0.001})

discriminator = get_discriminator(4, 40, 2, 1)
discriminator.cuda()

import torch.nn.init as init
for name, param in discriminator.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)
        
optimizer2 = optim.Adam(discriminator.parameters(), lr=5e-3,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)

nIter1 = 10000

loss_all_2 = []
test_loss_2 = []
C1_b_list = []
C2_b_list = []
C3_b_list = []

loss1_value = 1
it = 0


while  it<20000:
    
    if it <=1000:    
    
        ############ loss D ###########
        pre_H = PINNs2(train_t)

        d_fake = discriminator(torch.cat((train_t,pre_H.detach()),1))
        d_real = discriminator(torch.cat((train_t,train_data),1))

        loss_d = torch.mean(1-d_real)+torch.mean(d_fake)

        optimizer2.zero_grad()
        loss_d.backward()
        optimizer2.step()  

        ####### loss G#######
        pre_H = PINNs2(train_t)  

        d_fake = discriminator(torch.cat((train_t,pre_H),1))

        loss_L = torch.mean(1-d_fake)+torch.mean(torch.square(pre_H - train_data))

        optimizer1.zero_grad()
        loss_L.backward()
        optimizer1.step()     
    
    ##### loss_Bi  ######
    E_bound = PINNs2(bound_t)
    loss_bound = torch.mean(torch.square(E_bound[:,0:1]-real_bound[:,0:1]))+\
                 torch.mean(torch.square(E_bound[:,1:2]-real_bound[:,1:2]))+\
                 torch.mean(torch.square(E_bound[:,2:3]-real_bound[:,2:3]))
    
    ##### loss f  ######    
    E_inside = PINNs2(inside_t)
    E_inside_x = E_inside[:,0:1]
    E_inside_y = E_inside[:,1:2]   
    E_inside_z = E_inside[:,2:3]   
    
    d_x_t = autograd.grad(outputs=E_inside_x, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    
    d_y_t = autograd.grad(outputs=E_inside_y, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    
    d_z_t = autograd.grad(outputs=E_inside_z, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_z),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]      

    loss_f = torch.mean(torch.square(d_x_t-(C1_b*(E_inside_y-E_inside_x))))+\
             torch.mean(torch.square(d_y_t-(E_inside_x*(C2_b-E_inside_z)-E_inside_y)))+\
             torch.mean(torch.square(d_z_t-(E_inside_x*E_inside_y-C3_b*E_inside_z)))
    
    ##### loss observation  ######        
    E_observation = PINNs2(train_t)
    
    loss_observation = torch.mean(torch.square(E_observation[:,0:1]-train_data[:,0:1]))+\
                       torch.mean(torch.square(E_observation[:,1:2]-train_data[:,1:2]))+\
                       torch.mean(torch.square(E_observation[:,2:3]-train_data[:,2:3]))

    loss = loss_bound+loss_f+2*loss_observation
    
    loss_all_2.append(loss.item())
    loss1_value = loss.item()
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    if (it+1) % 1000 == 0 or it==0:
        print('It:', it, 'Loss:', loss.item())

        N_inside = 500
        N_bound = 10
        inside_t = lhs(1,N_inside)*(3-0)
        bound_t = np.zeros((N_bound,1))
        real_bound = np.concatenate((-8*np.ones((N_bound,1)),7*np.ones((N_bound,1)),27*np.ones((N_bound,1))),axis=1)

        inside_t = torch.from_numpy(inside_t).float()
        bound_t = torch.from_numpy(bound_t).float()
        real_bound = torch.from_numpy(real_bound).float()

        inside_t.requires_grad_()

        ###########GPU###########
        inside_t = inside_t.cuda()
        bound_t = bound_t.cuda()
        real_bound = real_bound.cuda()
        ###########GPU###########
        
        C1_b_list.append(C1_b.detach().item())
        C2_b_list.append(C2_b.detach().item())
        C3_b_list.append(C3_b.detach().item())
        print('C1:',C1_b,'C2:',C2_b,'C3:',C3_b)
        
    it = it + 1        
loss1_value    

It: 0 Loss: 1496.553955078125
C1: tensor(0.9990, requires_grad=True) C2: tensor(0.9990, requires_grad=True) C3: tensor(0.9990, requires_grad=True)
It: 999 Loss: 135.3621368408203
C1: tensor(0.3058, requires_grad=True) C2: tensor(1.8796, requires_grad=True) C3: tensor(0.2083, requires_grad=True)
It: 1999 Loss: 116.08294677734375
C1: tensor(0.4915, requires_grad=True) C2: tensor(3.1844, requires_grad=True) C3: tensor(0.1033, requires_grad=True)
It: 2999 Loss: 107.44810485839844
C1: tensor(0.6228, requires_grad=True) C2: tensor(4.4844, requires_grad=True) C3: tensor(0.0641, requires_grad=True)
It: 3999 Loss: 102.51361083984375
C1: tensor(0.7163, requires_grad=True) C2: tensor(5.6723, requires_grad=True) C3: tensor(0.0540, requires_grad=True)
It: 4999 Loss: 98.89891052246094
C1: tensor(0.7852, requires_grad=True) C2: tensor(6.7985, requires_grad=True) C3: tensor(0.0590, requires_grad=True)
It: 5999 Loss: 95.25296020507812
C1: tensor(0.8573, requires_grad=True) C2: tensor(7.8961, requires_g

0.015863755717873573

## Results

### Save data

In [1]:
# np.save('../experimental_data/J2/C1_a_list',C1_a_list)
# np.save('../experimental_data/J2/C1_b_list',C1_b_list)
# np.save('../experimental_data/J2/C2_a_list',C2_a_list)
# np.save('../experimental_data/J2/C2_b_list',C2_b_list)
# np.save('../experimental_data/J2/C3_a_list',C3_a_list)
# np.save('../experimental_data/J2/C3_b_list',C3_b_list)